In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import matplotlib as mpl

mpl.rcParams["figure.dpi"] = 1000
# mpl.rcParams['legend.fontsize'] = 15
# params = {'legend.fontsize': 20,
#           'legend.handlelength': 2}
# plot.rcParams.update(params)

In [3]:
from algorithms.moo.nsga2 import NSGA2
from constraints.as_obj import ConstraintsAsObjective
from operators.survival.rank_and_crowding.classes import (
    RankAndCrowding,
    MyConstrRankAndCrowding,
    ParallelConstrRankAndCrowding,
    MyConstrRankAndCrowding2,
)
from optimize import minimize
from visualization.scatter import Scatter
from pymoo.problems import get_problem

In [4]:
core_moea1 = NSGA2(pop_size=300, survival=RankAndCrowding(crowding_func="cd"),
                   eliminate_duplicates=True)
core_moea2 = NSGA2(
    pop_size=300, survival=MyConstrRankAndCrowding(crowding_func="cd"), eliminate_duplicates=True)
core_moea3 = NSGA2(
    pop_size=300, survival=ParallelConstrRankAndCrowding(), eliminate_duplicates=True)
core_moea4 = NSGA2(
    pop_size=300, survival=MyConstrRankAndCrowding2(), eliminate_duplicates=True)

In [5]:
import time


def performance_timer(func):
    """
    A decorator to measure and print the execution time of a function.
    """

    def wrapper(*args, **kwargs):
        start_time = time.time()  # Capture the start time
        result = func(*args, **kwargs)  # Execute the function
        end_time = time.time()  # Capture the end time
        duration = end_time - start_time  # Calculate duration
        # print(f"{func.__name__} executed in {duration:.6f} seconds")
        return result

    return wrapper


@performance_timer
def minimize_(*args, **kwargs):
    return minimize(*args, **kwargs)

In [6]:
from pymoo.indicators.gd import GD
from pymoo.indicators.gd_plus import GDPlus
from pymoo.indicators.igd import IGD
from pymoo.indicators.igd_plus import IGDPlus
from pymoo.indicators.hv import HV

metric_classes = {
    "Generational Distance (GD)": GD,
    "Generational Distance Plus (GD+)": GDPlus,
    "Inverted Generational Distance (IGD)": IGD,
    "Inverted Generational Distance Plus (IGD+)": IGDPlus,
    # "Hypervolume": HV,
}


def performance_indicators(pf, results):
    # Pre-configuration for each metric, handle special cases like HV here.
    pf_calculators = [
        (
            metric_class(ref_point=np.array([0, 0]))
            if metric_name == "Hypervolume"
            else metric_class(pf)
        )
        for metric_name, metric_class in metric_classes.items()
    ]

    def cal_acc_metric(sols):
        results_matrix = np.empty(
            (len(sols), len(metric_classes)), dtype=object)

        for idx, sol in enumerate(sols):
            # Ensure `sol` is a 2D array with the correct shape [n_samples, n_features]
            sol = np.atleast_2d(sol)
            if sol.shape[0] == 1 and sol.shape[1] != len(pf):
                # This implies sol is a single sample with multiple features, which is the correct format
                pass
            elif sol.shape[1] == 1 and sol.shape[0] != len(pf):
                # This implies sol has the wrong orientation; it's many samples of a single feature
                sol = sol.T

            for metric_idx, calculator in enumerate(pf_calculators):
                try:
                    results_matrix[idx, metric_idx] = calculator(sol)
                except Exception as e:
                    # print(
                    #     f"Error calculating {list(metric_classes.keys())[metric_idx]}: {e}")
                    results_matrix[idx, metric_idx] = np.nan

        return [
            (np.mean(results_matrix[:, i]), np.median(results_matrix[:, i]))
            for i in range(len(pf_calculators))
        ]

    return [cal_acc_metric(rs) for rs in results]

In [7]:
from constraints.as_obj import CVRAsObjective, CDFAsObjective, CVRAsObjective2, CDFAsObjective2


@performance_timer
def compare_on(problem, n_gen=300):
    # Configuration for each optimization strategy
    configurations = [
        # (problem, core_moea1, ("n_gen", n_gen), {"seed": 1, "verbose": False}),
        # (ConstraintsAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        (CVRAsObjective(problem, weighted_constraint_vector=[ 2, 60,   1, 5,  3, 4]),
         core_moea2, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem, weighted_constraint_vector=3),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem, weighted_constraint_vector=3),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem, weighted_constraint_vector=3),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem, weighted_constraint_vector=1),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem, weighted_constraint_vector=1.5),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CVRAsObjective(problem, weighted_constraint_vector=2),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (CDFAsObjective(problem),
        #  core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (
        #     CVRAsObjective2(problem),
        #     core_moea1,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        # (
        #     CDFAsObjective2(problem),
        #     core_moea1,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        # (ConstraintsAsObjective(problem),
        #  core_moea2, ("n_gen", n_gen), {"seed": 1}),
        # (
        #     CVRAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        # (
        #     CVRAsObjective2(problem),
        #     core_moea4,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        # (
        #     CDFAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        # (
        #     CDFAsObjective2(problem),
        #     core_moea4,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
    ]

    results = []
    for config in configurations:
        result = minimize_(*config[:-1], **config[-1])
        results.append(result)
    return results


def test(problem_name, n_gen=300, *args):
    if "dascmop" in problem_name:
        problem = get_problem(problem_name, args[0])
    elif "modact" in problem_name:
        problem = get_problem(problem_name, args[0])
    else:
        problem = get_problem(problem_name)
    pf = problem.pareto_front()
    # print(
    #     f"Problem {problem.name}: n_var={problem.n_var} n_obj={problem.n_obj} n_ieq_constr={problem.n_ieq_constr} n_eq_constr={problem.n_eq_constr}"
    # )

    results = compare_on(problem, n_gen=n_gen)

    # Check if any of the results is None
    if any(result is None for result in results):
        print("One or more optimization runs failed to return a result.")
        # Return None in place of results to indicate failure
        return pf, [None] * len(results)

    # Prepare data for performance indicators, safely handling cases where result.F might be None or incorrectly shaped
    prepared_data = []
    for i, result in enumerate(results):
        if result and hasattr(result, "F") and result.F is not None:
            # i == 0 this NGSA without additional objective
            if i in [0, 1, 2]:
                # if i == 1 or i == 3:
                # if True:
                prepared_data.append(result.F[:, :-1])
            else:
                # prepared_data.append(result.F[:, :-1])
                prepared_data.append(result.F[:, :-2])
        else:
            # Use None or an appropriate placeholder if result is invalid
            print(f'algo {i} not converge')
            prepared_data.append(None)

    # Ensure all data is valid before calculating performance indicators
    if all(data is not None for data in prepared_data):
        performance_results = performance_indicators(pf, prepared_data)
        for i, metric in enumerate(metric_classes.keys()):
            pass
            # print(
            #     f"{metric}: " +
            #     " | ".join(f"{perf[i]}" for perf in performance_results)
            # )
    else:
        print("Performance indicators could not be calculated due to invalid data.")

    return pf, *results

# BNH


In [8]:
pf, *results = test("bnh", n_gen=100)
print(len(results[0].F[results[0].F[:, -1] == 0]),
        # len(results[1].F[results[1].F[:, -1] == 0]),
        # len(results[2].F[results[2].F[:, -1] == 0]),
        # len(results[3].F[results[3].F[:, -1] == 0]),
        # len(results[4].F[results[4].F[:, -1] == 0])
        )

CVRAsObjective: weighted_constraint_vector=[2, 60, 1, 5, 3, 4]


ValueError: operands could not be broadcast together with shapes (300,2) (6,) 

In [ ]:
for i in range(1, 600, 200):
    # print(f"\n I = {i}")
    pf, *results = test("carside", n_gen=i)
    print(len(results[0].F[results[0].F[:, -1] == 0]),
          len(results[1].F[results[1].F[:, -1] == 0]),
          len(results[2].F[results[2].F[:, -1] == 0]),
          len(results[3].F[results[3].F[:, -1] == 0]),
          len(results[4].F[results[4].F[:, -1] == 0])
          )

In [ ]:
for i in range(1, 600, 200):
    # print(f"\n I = {i}")
    pf, *results = test("tnk", n_gen=i)
    print(len(results[0].F[results[0].F[:, -1] == 0]),
          len(results[1].F[results[1].F[:, -1] == 0]),
          len(results[2].F[results[2].F[:, -1] == 0]),
          len(results[3].F[results[3].F[:, -1] == 0]),
          len(results[4].F[results[4].F[:, -1] == 0])
          )

In [9]:
pf, *results = test("osy", n_gen=200)
print(len(results[0].F[results[0].F[:, -1] == 0]),
        # len(results[1].F[results[1].F[:, -1] == 0]),
        # len(results[2].F[results[2].F[:, -1] == 0]),
        # len(results[3].F[results[3].F[:, -1] == 0]),
        # len(results[4].F[results[4].F[:, -1] == 0])
        )

CVRAsObjective: weighted_constraint_vector=[2, 60, 1, 5, 3, 4]
Infeasible rate: 0.9733333333333334
[  8 249 116  23 176  81]
cvr mean: 53.46333333333333
Infeasible rate: 0.97
[  8 251 113  27 175  80]
cvr mean: 53.89666666666667
Infeasible rate: 0.94
[ 14 209  81  23 125  46]
cvr mean: 44.41
Infeasible rate: 0.8833333333333333
[ 28 133  69  22 102  65]
cvr mean: 29.27
Infeasible rate: 0.7333333333333333
[ 56  15  81  10 106  60]
cvr mean: 5.67
Infeasible rate: 0.5366666666666666
[57 16 55  4 37 14]
cvr mean: 4.386666666666667
Infeasible rate: 0.2
[14 12  9  3  6 22]
cvr mean: 2.9266666666666667
Infeasible rate: 0.18666666666666668
[11 11  1  3  7 26]
cvr mean: 2.743333333333333
Infeasible rate: 0.20666666666666667
[17  9  1  8  8 24]
cvr mean: 2.45
Infeasible rate: 0.28
[14 27  3 12 12 24]
cvr mean: 6.1433333333333335
Infeasible rate: 0.26
[10 25  2  9  2 36]
cvr mean: 5.723333333333334
Infeasible rate: 0.27666666666666667
[28 23  3  6  5 25]
cvr mean: 5.28
Infeasible rate: 0.293333333

KeyboardInterrupt: 

In [ ]:
pf, *results = test("osy", n_gen=200)
print(len(results[0].F[results[0].F[:, -1] == 0]),
        # len(results[1].F[results[1].F[:, -1] == 0]),
        # len(results[2].F[results[2].F[:, -1] == 0]),
        # len(results[3].F[results[3].F[:, -1] == 0]),
        # len(results[4].F[results[4].F[:, -1] == 0])
        )

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F[:, :-1], marker="+",
         color="yellow", label="UMOP + Baseline")
plot.show()

In [ ]:
for j in range(10, 17):
    print(f"\nLevel = {j}")
    # for i in range(1, 500, 10):
        # print(f"\n I = {i}")
    pf, *results = test("dascmop1", 200, j)
    print(len(results[0].F[results[0].F[:, -1] == 0]),
        len(results[1].F[results[1].F[:, -1] == 0]),
        len(results[2].F[results[2].F[:, -1] == 0]),
        # len(results[3].F[results[3].F[:, -1] == 0]),
        # len(results[4].F[results[4].F[:, -1] == 0])
        )

In [ ]:
for i in range(1, 100, 10):
    # print(f"\n I = {i}")
    pf, *results = test("dascmop1", i, 16)
    print(len(results[0].F[results[0].F[:, -1] == 0]),
          len(results[1].F[results[1].F[:, -1] == 0]),
          len(results[2].F[results[2].F[:, -1] == 0]),
          len(results[3].F[results[3].F[:, -2] == 0]),
          len(results[4].F[results[4].F[:, -2] == 0])
          )

In [ ]:
for i in range(1, 100, 10):
    # print(f"\n I = {i}")
    pf, *results = test("dascmop1", i, 1)
    print(len(results[0].F[results[0].F[:, -1] == 0]),
          len(results[1].F[results[1].F[:, -1] == 0]),
          len(results[2].F[results[2].F[:, -1] == 0]),
          len(results[3].F[results[3].F[:, -2] == 0]),
          len(results[4].F[results[4].F[:, -2] == 0])
          )

In [ ]:
for i in range(1, 100, 10):
    # print(f"\n I = {i}")
    pf, *results = test("dascmop1", i, 16)
    print(len(results[0].F[results[0].F[:, -1] == 0]),
          len(results[1].F[results[1].F[:, -1] == 0]),
          len(results[2].F[results[2].F[:, -1] == 0]),
          len(results[3].F[results[3].F[:, -2] == 0]),
          len(results[4].F[results[4].F[:, -2] == 0])
          )

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F[results[0].F[:, -1] == 0][:, :-1], marker="d",
         color="yellow", label="UMOP + Baseline")
# plot.add(results[2].F[:, :-1], marker="+",
#  color="green", label="UMOP + Baseline + Our CDP")
plot.add(
    results[1].F[results[1].F[:, -1] == 0][:, :-1], marker="1", color="blue", label="UMOP + Our generalized framework"
)
plot.add(results[2].F[results[2].F[:, -1] == 0][:, :-1], marker="2",
         color="purple", label="UMOP + Our framework + CDF")
plot.add(results[3].F[results[3].F[:, -1] == 0][:, :-2], marker="2",
         color="green", label="UMOP + Our framework + CDF")
plot.add(results[4].F[results[4].F[:, -1] == 0][:, :-2], marker="2",
         color="black", label="UMOP + Our framework + CDF")
plot.show()

In [ ]:
results[4].F

In [ ]:
pf, *results = test("g12", n_gen=100)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g13", n_gen=100)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g14", n_gen=1000)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g15", n_gen=1000)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g16", n_gen=1000)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g17", n_gen=1000)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g18", n_gen=200)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g19", n_gen=1000)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g20", n_gen=2000)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g21", n_gen=2000)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g22", n_gen=2000)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g23", n_gen=1000)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test("g24", n_gen=200)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
pf, *results = test(("dascmop9", 1), n_gen=200)
pf, results[0].F, results[1].F, results[2].F

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
# plot.add(results[0].F, marker="x", color="red", label="Original CMOP + NSGA-II")
plot.add(results[0].F[:, :-1], marker="d",
         color="yellow", label="UMOP + Baseline")
# plot.add(results[2].F[:, :-1], marker="+",
#  color="green", label="UMOP + Baseline + Our CDP")
plot.add(
    results[1].F[:, :-1], marker="1", color="blue", label="UMOP + Our generalized framework"
)
plot.add(results[2].F[:, :-1], marker="2",
         color="purple", label="UMOP + Our framework + CDF")
plot.show()

# SRN


In [ ]:
pf, *results = test("srn", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
# plot.add(results[0].F, marker="x", color="red", label="Original CMOP + NSGA-II")
plot.add(results[0].F[:, :-1], marker="d",
         color="yellow", label="UMOP + Baseline")
# plot.add(results[2].F[:, :-1], marker="+",
#  color="green", label="UMOP + Baseline + Our CDP")
plot.add(
    results[1].F[:, :-1], marker="1", color="blue", label="UMOP + Our generalized framework"
)
plot.add(results[2].F[:, :-1], marker="2",
         color="purple", label="UMOP + Our framework + CDF")
plot.show()

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange")
plot.add(results[0].F, marker="x", color="red")
plot.add(results[1].F[results[1].F[:, -1] == 0]
         [:, :-1], marker="d", color="yellow")
plot.add(results[2].F[:, :-1], marker="+",
         color="green")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue"
)
plot.add(results[4].F[:, :-1], marker="2",
         color="purple")
plot.show()

# ZDT


In [ ]:
# problem_zdt1 = get_problem("zdt1")
# problem = problem_zdt1
# print(
#     f"Problem {problem.name}: n_var={problem.n_var} n_obj={problem.n_obj} n_ieq_constr={problem.n_ieq_constr} n_eq_constr={problem.n_eq_constr}"
# )

# res0, res1, res2, res3 = compare_on(problem)
# print(
#     f"F1-F2: {sum(res1.F - res2.F)}   F2-F3{sum(res2.F - res3.F)}   F1-F3{sum(res1.F - res3.F)}"
# )

# plot = Scatter()
# plot.add(problem.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt2 = get_problem("zdt2")

# res0, res1, res2, res3 = compare_on(problem_zdt2)

# plot = Scatter()
# plot.add(problem_zdt2.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt3 = get_problem("zdt3")

# res0, res1, res2, res3 = compare_on(problem_zdt3)

# plot = Scatter()
# plot.add(problem_zdt3.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.Fa, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt4 = get_problem("zdt4")

# res0, res1, res2, res3 = compare_on(problem_zdt4)
# print(
#     f"F1-F2: {sum(res1.F - res2.F)}   F2-F3{sum(res2.F - res3.F)}   F1-F3{sum(res1.F - res3.F)}"
# )

# plot = Scatter()
# plot.add(problem_zdt4.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt5 = get_problem("zdt5", normalize=False)

# res0, res1, res2, res3 = compare_on(problem_zdt5, n_gen=3000)
# print(
#     f"F1-F2: {sum(res1.F - res2.F)}   F2-F3{sum(res2.F - res3.F)}   F1-F3{sum(res1.F - res3.F)}"
# )

# plot = Scatter()
# plot.add(problem_zdt5.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# problem_zdt6 = get_problem("zdt6")

# res0, res1, res2, res3 = compare_on(problem_zdt6)
# print(
#     f"F1-F2: {sum(res1.F - res2.F)}   F2-F3{sum(res2.F - res3.F)}   F1-F3{sum(res1.F - res3.F)}"
# )

# plot = Scatter()
# plot.add(problem_zdt6.pareto_front(), marker="*", color="orange", label="Pareto front")
# plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline CV_as_obj")
# plot.add(res2.F[:, :-1], marker="1", color="blue", label="Our CV_as_obj")
# plot.add(res3.F[:, :-1], marker="1", color="purple", label="Our CV_as_obj")
# plot.show()

In [ ]:
# cv = res3.F[:, 0]
# least_infeas = cv.argmin()
# x = res3.X[least_infeas]

# sol = Individual(X=x)
# Evaluator().eval(problem2, sol)

# print("Best solution found: \nX = %s\nF = %s\nCV = %s" % (sol.X, sol.F, sol.CV))

In [ ]:
# plot = Scatter()
# plot.add(problem2.pareto_front(), marker="*", color="black", alpha=0.7, s=100)
# plot.add(res3.F[:,1:], facecolor="none", edgecolor="red")
# plot.show()

In [ ]:
# X, F = res3.opt.get("X", "F")
# hist = res3.history
# print(len(hist))

In [ ]:
# n_evals = []             # corresponding number of function evaluations\
# hist_F = []              # the objective space values in each generation
# hist_cv = []             # constraint violation in each generation
# hist_cv_avg = []         # average constraint violation in the whole population

# for algo in hist:

#     # store the number of function evaluations
#     n_evals.append(algo.evaluator.n_eval)

#     # retrieve the optimum from the algorithm
#     opt = algo.opt

#     # store the least contraint violation and the average in each population
#     hist_cv.append(opt.get("CV").min())
#     hist_cv_avg.append(algo.pop.get("CV").mean())

#     # filter out only the feasible and append and objective space values
#     feas = np.where(opt.get("feasible"))[0]
#     hist_F.append(opt.get("F")[feas])

In [ ]:
# k = np.where(np.array(hist_cv) <= 0.0)[0].min()
# print(f"At least one feasible solution in Generation {k} after {n_evals[k]} evaluations.")

In [ ]:
# # replace this line by `hist_cv` if you like to analyze the least feasible optimal solution and not the population
# vals = hist_cv_avg

# k = np.where(np.array(vals) <= 0.0)[0].min()
# print(f"Whole population feasible in Generation {k} after {n_evals[k]} evaluations.")

# plt.figure(figsize=(7, 5))
# plt.plot(n_evals, vals,  color='black', lw=0.7, label="Avg. CV of Pop")
# plt.scatter(n_evals, vals,  facecolor="none", edgecolor='black', marker="p")
# plt.axvline(n_evals[k], color="red", label="All Feasible", linestyle="--")
# plt.title("Convergence")
# plt.xlabel("Function Evaluations")
# plt.ylabel("Hypervolume")
# plt.legend()
# plt.show()

In [ ]:
# approx_ideal = F[:,-2:].min(axis=0)
# approx_nadir = F[:,-2:].max(axis=0)
# approx_ideal, approx_nadir

In [ ]:
# from pymoo.indicators.hv import Hypervolume

# metric = Hypervolume(ref_point=np.array([0.1, 0.1]),
#                      norm_ref_point=False,
#                      zero_to_one=True,
#                      ideal=approx_ideal,
#                      nadir=approx_nadir)

# hv = [metric.do(_F[:, -2:]) for _F in hist_F]

# plt.figure(figsize=(7, 5))
# plt.plot(n_evals, hv, color='black', lw=0.7, label="Avg. CV of Pop")
# plt.scatter(n_evals, hv, facecolor="none", edgecolor='black', marker="p")
# plt.title("Convergence")
# plt.xlabel("Function Evaluations")
# plt.ylabel("Hypervolume")
# plt.show()

In [ ]:
# from pymoo.util.running_metric import RunningMetricAnimation

# running = RunningMetricAnimation(delta_gen=5,
#                         n_plots=3,
#                         key_press=False,
#                         do_show=True)

# for algorithm in res3.history[:50]:
#     running.update(algorithm)

In [ ]:
# from pymoo.indicators.igd import IGD

# metric = IGD(pf, zero_to_one=True)

# igd = [metric.do(_F) for _F in hist_F]

# plt.plot(n_evals, igd,  color='black', lw=0.7, label="Avg. CV of Pop")
# plt.scatter(n_evals, igd,  facecolor="none", edgecolor='black', marker="p")
# plt.axhline(10**-2, color="red", label="10^-2", linestyle="--")
# plt.title("Convergence")
# plt.xlabel("Function Evaluations")
# plt.ylabel("IGD")
# plt.yscale("log")
# plt.legend()
# plt.show()

# OSY


In [ ]:
pf, *results = test("osy", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange")
plot.add(results[0].F, marker="x", color="red")
plot.add(results[1].F[results[1].F[:, -1] == 0]
         [:, :-1], marker="d", color="yellow")
plot.add(results[2].F[:, :-1], marker="+",
         color="green")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue"
)
plot.add(results[4].F[:, :-1], marker="2",
         color="purple")
plot.show()

# TNK


In [ ]:
pf, *results = test("tnk", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
# plot.add(results[0].F, marker="x", color="red", label="Original CMOP + NSGA-II")
plot.add(results[0].F[:, :-1], marker="d",
         color="yellow", label="UMOP + Baseline")
# plot.add(results[2].F[:, :-1], marker="+",
#  color="green", label="UMOP + Baseline + Our CDP")
plot.add(
    results[1].F[:, :-1], marker="1", color="blue", label="UMOP + Our generalized framework"
)
plot.add(results[2].F[:, :-1], marker="2",
         color="purple", label="UMOP + Our framework + CDF")
plot.show()

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange")
plot.add(results[0].F, marker="x", color="red")
plot.add(results[1].F[results[1].F[:, -1] == 0]
         [:, :-1], marker="d", color="yellow")
plot.add(results[2].F[:, :-1], marker="+",
         color="green")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue"
)
plot.add(results[4].F[:, :-1], marker="2",
         color="purple")
plot.show()

In [ ]:
# plt.hist(res1.F[:, -1:], bins=30, color="green", alpha=0.7, label="Sum CV_as_obj")
# plt.hist(res2.F[:, -1:], bins=30, color="blue", alpha=0.7, label="Max CV_as_obj")
# plt.hist(res3.F[:, -1:], bins=30, color="purple", alpha=0.7, label="Median CV_as_obj")

# Add labels and title
# plt.xlabel("Values")
# plt.ylabel("Frequency")
# plt.title("Histogram of CV")

# Add a legend
# plt.legend()

# Show the plot
# plt.show()

# Carside


In [ ]:
pf, *results = test("Carside", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
# plot.add(results[0].F, marker="x", color="red", label="Original CMOP + NSGA-II")
plot.add(results[0].F[results[0].F[:, -1] == 0][:, :-1],
         marker="d", color="yellow", label="UMOP + Baseline")
# plot.add(results[2].F[:, :-1], marker="+",
#  color="green", label="UMOP + Baseline + Our CDP")
plot.add(
    results[1].F[results[1].F[:, -1] == 0][:, :-1], marker="1", color="blue", label="UMOP + Our generalized framework"
)
plot.add(results[2].F[results[2].F[:, -1] == 0][:, :-1], marker="2",
         color="purple", label="UMOP + Our framework + CDF")
plot.show()

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red",
         label="Original CMOP + NSGA-II")
plot.add(results[1].F[results[1].F[:, -1] == 0][:, :-1],
         marker="d", color="yellow", label="UMOP + Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="UMOP + Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="UMOP + Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="2",
         color="purple", label="UMOP + Our framework + CDF")
plot.show()

In [ ]:
plot = Scatter(tight_layout=True)
# plot.add(results[1].F, marker="+", color="green", label="Baseline")
plot.add(results[2].F, marker=".", color="green", label="Baseline + Our CDP")
plot.add(results[3].F, marker="1", color="blue", label="Our generalized framework")
plot.add(results[4].F, marker="1", color="purple", label="Our framework + CDF")
plot.show()

# Pressure Vessel


In [ ]:
pf, *results = test("pressure_vessel", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

# G


## G1


In [ ]:
pf, *results = test("g1", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
plot.add(results[1].F[results[1].F[:, -1] == 0][:, :-1],
         marker="D", color="Yellow", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+",
         color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple", label="Our framework + CDF")
plot.show()

## G2


In [ ]:
pf, *results = test("g1", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange")
plot.add(results[0].F, marker="x", color="red")
plot.add(results[1].F[results[1].F[:, -1] == 0]
         [:, :-1], marker="D", color="yellow")
plot.add(results[2].F[:, :-1], marker="+",
         color="green")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue"
)
plot.add(results[4].F[:, :-1], marker="1",
         color="purple")
plot.show()

## G3 (1 Constraint)


In [ ]:
# pf, res0, res1, res2, res3 = test("g3", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G4


In [ ]:
pf, res0, res1, res2, res3 = test("g4", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G5


In [ ]:
pf, res0, res1, res2, res3 = test("g5", n_gen=300)

In [ ]:
pf

In [ ]:
res0.F

In [ ]:
res2.F

In [ ]:
res3.F

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G6


In [ ]:
pf, res0, res1, res2, res3 = test("g6", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G7


In [ ]:
pf, res0, res1, res2, res3 = test("g7", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G8


In [ ]:
pf, res0, res1, res2, res3 = test("g8", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G9


In [ ]:
pf, res0, res1, res2, res3 = test("g9", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G10


In [ ]:
pf, res0, res1, res2, res3 = test("g10", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G11


In [ ]:
pf, res0, res1, res2, res3 = test("g11", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G12


In [ ]:
pf, res0, res1, res2, res3 = test("g12", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G13


In [ ]:
pf, res0, res1, res2, res3 = test("g13", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

## G14


In [ ]:
pf, res0, res1, res2, res3 = test("g14", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(res0.F, marker="x", color="red", label="Death penalty")
# plot.add(res1.F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(res2.F[:, :-1], marker="1", color="blue",
         label="Our generalized framework")
plot.add(res3.F[:, :-1], marker="1", color="purple",
         label="Our framework + CDF")
plot.show()

# MW


## MW1 (1 constraint)


In [ ]:
pf, *results = test("mw", n_gen=300)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW2


In [ ]:
pf, *results = test("mw2", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW3


In [ ]:
pf, *results = test("mw3", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW4 (1 constraint)


In [ ]:
pf, *results = test("mw4", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW5


In [ ]:
pf, *results = test("mw5", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW6 (1 constraint)


In [ ]:
pf, *results = test("mw6", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW7


In [ ]:
pf, *results = test("mw7", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW8 (1 constraint)


In [ ]:
pf, *results = test("mw8", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW9


In [ ]:
pf, *results = test("mw9", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW10


In [ ]:
pf, *results = test("mw13", n_gen=15)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

## MW14 (1 constraint)


In [ ]:
pf, *results = test("mw14", n_gen=100)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
# plot.add(results[3].F[:, :-1], marker="1", color="blue",
#          label="Our generalized framework")
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

# DAS-CMOP


## DAS-CMOP-1


In [ ]:
pf, *results = test("dascmop1", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop2", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop3", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop4", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop5", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop6", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop7", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop8", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
pf, *results = test("dascmop9", n_gen=500)

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

# MODAct


In [ ]:
pf, *results = test("modact", 10, "cs1")

In [ ]:
plot = Scatter()
plot.add(pf, marker="*", color="orange", label="Pareto front")
plot.add(results[0].F, marker="x", color="red", label="Death penalty")
# plot.add(results[1].F[:, :-1], marker="+", color="green", label="Baseline")
plot.add(results[2].F[:, :-1], marker="+", color="green", label="Baseline + Our CDP")
plot.add(
    results[3].F[:, :-1], marker="1", color="blue", label="Our generalized framework"
)
plot.add(results[4].F[:, :-1], marker="1", color="purple", label="Our framework + CDF")
plot.show()

In [ ]:
terminate_here

# To_excel


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from constraints.as_obj import CVRAsObjective, CDFAsObjective

POP_SIZE = 500
N_GEN = 300

core_moea1 = NSGA2(pop_size=POP_SIZE, survival=RankAndCrowding())
core_moea2 = NSGA2(pop_size=POP_SIZE, survival=MyConstrRankAndCrowding())
core_moea3 = NSGA2(pop_size=POP_SIZE, survival=ParallelConstrRankAndCrowding())


@performance_timer
def compare_on(problem, n_gen=300):
    # Configuration for each optimization strategy
    configurations = [
        # (problem, core_moea1, ("n_gen", n_gen), {"seed": 1, "verbose": False}),
        (ConstraintsAsObjective(problem),
         core_moea1, ("n_gen", n_gen), {"seed": 1}),
        (CVRAsObjective(problem),
         core_moea1, ("n_gen", n_gen), {"seed": 1}),
        (CDFAsObjective(problem),
         core_moea1, ("n_gen", n_gen), {"seed": 1}),
        # (ConstraintsAsObjective(problem),
        #  core_moea2, ("n_gen", n_gen), {"seed": 1}),
        # (
        #     CVRAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
        # (
        #     CDFAsObjective(problem),
        #     core_moea2,
        #     ("n_gen", n_gen),
        #     {"seed": 1, "verbose": False},
        # ),
    ]

    results = []
    for config in configurations:
        result = minimize_(*config[:-1], **config[-1])
        results.append(result)
    return results


def test(problem_name, n_gen=300, **kwargs):
    if "dascmop" in problem_name or "modact" in problem_name:
        if 'additional_args' in kwargs:
            problem = get_problem(problem_name, kwargs['additional_args'])
        else:
            raise ValueError(
                f"Additional arguments required for problem {problem_name}")
    else:
        problem = get_problem(problem_name)
    pf = problem.pareto_front()
    print(f"Problem {problem.name}: n_var={problem.n_var} n_obj={problem.n_obj} n_ieq_constr={problem.n_ieq_constr} n_eq_constr={problem.n_eq_constr}")

    results = compare_on(problem, n_gen=n_gen)

    # Check if any of the results is None
    if any(result is None for result in results):
        print("One or more optimization runs failed to return a result.")
        return pf, [None]*len(results)

    # Prepare data for performance indicators
    prepared_data = []
    for i, result in enumerate(results):
        if result and hasattr(result, 'F') and result.F is not None:
            # if i > 0:
            if True:
                prepared_data.append(result.F[:, :-1])
            else:
                prepared_data.append(result.F)
        else:
            prepared_data.append(None)

    # Ensure all data is valid before calculating performance indicators
    if all(data is not None for data in prepared_data):
        performance_results = performance_indicators(pf, prepared_data)
        for i, metric in enumerate(metric_classes.keys()):
            print(f"{metric}: " +
                  " | ".join(f"{perf[i]}" for perf in performance_results))
    else:
        print("Performance indicators could not be calculated due to invalid data.")

    return pf, results


def create_charts(df, output_dir='.'):
    metrics = df['Metric'].unique()
    problems = df['Problem'].unique()

    for problem in problems:
        problem_df = df[df['Problem'] == problem]
        for metric in metrics:
            metric_df = problem_df[problem_df['Metric'] == metric]
            plt.figure(figsize=(10, 6))
            plt.bar(metric_df['Algorithm'],
                    metric_df['GD+ Mean'], yerr=metric_df['GD+ Median'])
            plt.title(f'{metric} for {problem}')
            plt.xlabel('Algorithm')
            plt.ylabel(metric)
            plt.savefig(f'{output_dir}/{problem}_{metric}.png')
            plt.close()

# Function to compare multiple problems and export results to Excel


def compare_problems_to_excel(problem_configs, n_gen=300, output_file='results.xlsx'):
    results_list = []
    for problem_config in problem_configs:
        additional_args = None
        if isinstance(problem_config, tuple):
            problem_name, additional_args = problem_config
            pf, results = test(problem_name, n_gen=n_gen,
                               additional_args=additional_args)
        else:
            problem_name = problem_config
            pf, results = test(problem_name, n_gen=n_gen)

        # Skip problems where results are None
        if results[0] is None:
            continue

        # Compute performance indicators
        prepared_data = []
        for i, result in enumerate(results):
            if result and hasattr(result, 'F') and result.F is not None:
                # if i > 0:
                if True:
                    prepared_data.append(result.F[:, :-1])
                else:
                    prepared_data.append(result.F)
            else:
                prepared_data.append(None)

        if all(data is not None for data in prepared_data):
            performance_results = performance_indicators(pf, prepared_data)
            for i, rs in enumerate(performance_results):
                (gd_mean, gd_median), (gd_plus_mean, gd_plus_median), (igd_mean,
                                                                       igd_median), (igd_plus_mean, igd_plus_median) = rs
                result_row = {
                    "Problem": problem_name if additional_args is None else f'{problem_name} {additional_args}',
                    "Algorithm": f"Algorithm {i+1}",
                    "GD Mean": gd_mean,
                    "GD Median": gd_median,
                    "GD+ Mean": gd_plus_mean,
                    "GD+ Median": gd_plus_median,
                    "IGD Mean": igd_mean,
                    "IGD Median": igd_median,
                    "IGD+ Mean": igd_plus_mean,
                    "IGD+ Median": igd_plus_median,
                    "Time": results[i].exec_time if hasattr(results[i], 'exec_time') else np.nan
                }
                results_list.append(result_row)

    # Create DataFrame and export to Excel
    df = pd.DataFrame(results_list)
    df.to_excel(output_file, index=False)
    print(f"Results exported to {output_file}")

    # Create charts for visualizing the results
    # create_charts(df)


# Example usage
problem_configs = [
    # "wrm",
    # "srn",
    # "clutch"
]

problem_dascmop = [f'dascmop{i}' for i in range(1, 10)]
for i in range(16, 17):
    for p in problem_dascmop:
        problem_configs.append((p, i))
compare_problems_to_excel(problem_configs, n_gen=100,
                          output_file=f'optimization_results_{datetime.now().timestamp()}.xlsx')

In [ ]:
problem_configs = [
    "g7",
    "g8",
    "g9",
    # "mw7",
    # "mw7",
    # "mw9",
    # "mw10",
]
# problem_configs += [f'g{i}' for i in range(1,25)]
compare_problems_to_excel(
    problem_configs,
    n_gen=500,
    output_file=f"optimization_results_{datetime.now().timestamp()}.xlsx",
)

# Diversity Factor


In [ ]:
import numpy as np
from pymoo.core.problem import Problem
from pymoo.problems import get_problem
from scipy.spatial import cKDTree
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SimulatedBinaryCrossover as SBX
from pymoo.operators.mutation.pm import PolynomialMutation as PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.optimize import minimize
from pymoo.core.callback import Callback

In [ ]:
from annoy import AnnoyIndex


class DiversityEnhancedProblem(Problem):
    def __init__(self, problem, neighbors=5, trees=10):
        super().__init__(
            n_var=problem.n_var,
            n_obj=problem.n_obj + 1,
            n_constr=problem.n_constr,
            xl=problem.xl,
            xu=problem.xu,
        )
        self.base_problem = problem
        self.neighbors = neighbors
        self.population = []
        self.tree = AnnoyIndex(self.n_var, "manhattan")
        self.tree_cache = {}  # To store diversity values
        self.trees = trees  # Number of trees for Annoy

    def update_population(self, pop):
        self.population = pop.get("X")
        self.tree.unbuild()  # Clear the current tree
        for i, x in enumerate(self.population):
            self.tree.add_item(i, x.tolist())
        self.tree.build(self.trees)
        self.tree_cache = {}  # Clear cache after update

    def calculate_diversity(self, x):
        if len(self.population) > 1:
            distances = self.tree.get_nns_by_vector(
                x, self.neighbors, include_distances=True
            )[1]
            return 1.0 / np.mean(distances)
        return 0

    def _evaluate(self, x, out, *args, **kwargs):
        base_out = {}
        self.base_problem._evaluate(x, base_out, *args, **kwargs)
        f = base_out["F"]
        x = np.atleast_2d(x)

        # Calculate diversity, using cache if available
        diversity = np.array(
            [self.tree_cache.get(tuple(xi), self.calculate_diversity(xi)) for xi in x]
        )
        for xi, div in zip(x, diversity):
            self.tree_cache[tuple(xi)] = div  # Update cache

        out["F"] = np.column_stack([f, diversity])


class DiversityCallback(Callback):
    def __init__(self, problem):
        super().__init__()
        self.problem = problem

    def notify(self, algorithm):
        self.problem.update_population(algorithm.pop)


# Original problem from pymoo
original_problem = get_problem("zdt2")

# Enhanced problem with diversity objective
problem = DiversityEnhancedProblem(original_problem, neighbors=10)

# Setup the algorithm
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True,
)

# Execute the optimization
res1 = minimize(
    problem,
    algorithm,
    ("n_gen", 10000),
    verbose=True,
    seed=1,
    callback=DiversityCallback(problem),
)

res2 = minimize(
    original_problem,
    algorithm,
    ("n_gen", 2000),
    verbose=False,
    seed=0,
    # callback=DiversityCallback(problem)
)

In [ ]:
print(res1.F.shape, res2.F.shape)

In [ ]:
from visualization.scatter import Scatter

plot = Scatter()
plot.add(res1.F[:, :-1], facecolor="none", edgecolor="red")
plot.add(res2.F, facecolor="none", edgecolor="blue")
plot.show()

In [ ]:
from visualization.scatter import Scatter

plot = Scatter()
plot.add(res1.F, facecolor="none", edgecolor="red")
# plot.add(res2.F, facecolor="none", edgecolor="blue")
plot.show()

In [ ]:
from pymoo.factory import get_performance_indicator

# Assuming 'result' is the output of your optimization algorithm
hv = get_performance_indicator(
    "hv", ref_point=np.array([1.1, 1.1])
)  # Adjust ref_point as needed
hypervolume = hv.calc(res1.F)

print("Hypervolume:", hypervolume)

In [ ]:
from visualization.scatter import Scatter
from annoy import AnnoyIndex
import numpy as np


class DiversityEnhancedProblem2(Problem):
    def __init__(
        self, problem, neighbors=5, trees=10, update_interval=5, change_threshold=0.2
    ):
        super().__init__(
            n_var=problem.n_var,
            n_obj=problem.n_obj + 1,
            n_constr=problem.n_constr,
            xl=problem.xl,
            xu=problem.xu,
        )
        self.base_problem = problem
        self.neighbors = neighbors
        self.population = []
        self.tree = AnnoyIndex(self.n_var, "euclidean")
        self.trees = trees
        self.update_interval = update_interval
        self.change_threshold = change_threshold
        self.generation_count = 0
        self.last_population = None

    def update_population(self, pop):
        self.population = pop.get("X")
        self.generation_count += 1

        # Determine if the tree needs updating
        if (
            self.last_population is None
            or self.generation_count % self.update_interval == 0
            or self.population_has_changed()
        ):
            self.rebuild_tree()
            self.last_population = np.array(self.population.copy())
            print("Tree rebuilt.")

    def population_has_changed(self):
        """Checks if significant changes have occurred in the population"""
        if self.last_population is not None:
            dists = np.linalg.norm(
                self.last_population - self.population, axis=1)
            significant_changes = np.mean(dists > self.change_threshold)
            # More than 10% of the population has significantly changed
            return significant_changes > 0.1
        return False

    def rebuild_tree(self):
        """Rebuilds the Annoy index"""
        self.tree.unbuild()
        for i, x in enumerate(self.population):
            self.tree.add_item(i, x.tolist())
        self.tree.build(self.trees)

    def calculate_diversity(self, x):
        if len(self.population) > 1:
            distances = self.tree.get_nns_by_vector(
                x, self.neighbors, include_distances=True
            )[1]
            return 1.0 / np.mean(distances)
        return 0

    def _evaluate(self, x, out, *args, **kwargs):
        base_out = {}
        self.base_problem._evaluate(x, base_out, *args, **kwargs)
        f = base_out["F"]
        x = np.atleast_2d(x)

        # Calculate diversity
        diversity = np.array([self.calculate_diversity(xi) for xi in x])
        out["F"] = np.column_stack([f, diversity])


# Original problem from pymoo
original_problem = get_problem("zdt2")

# Enhanced problem with diversity objective
problem2 = DiversityEnhancedProblem2(original_problem, neighbors=10)

# Setup the algorithm
algorithm = NSGA2(
    pop_size=100,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True,
)

# Execute the optimization
res1 = minimize(
    problem2,
    algorithm,
    ("n_gen", 5000),
    verbose=True,
    seed=1,
    callback=DiversityCallback(problem),
)

res2 = minimize(
    original_problem,
    algorithm,
    ("n_gen", 2000),
    verbose=False,
    seed=0,
)


plot = Scatter()
plot.add(res1.F[:, 1:], facecolor="none", edgecolor="red")
plot.add(res2.F, facecolor="none", edgecolor="blue")
plot.show()

In [ ]:
from pymoo.core.problem import Problem
import numpy as np


class ConstraintAsObjectiveProblem(Problem):
    def __init__(self, problem):
        super().__init__(
            n_var=problem.n_var,
            n_obj=problem.n_obj + problem.n_constr,
            n_constr=0,  # No explicit constraints as they are now objectives
            xl=problem.xl,
            xu=problem.xu,
        )
        self.problem = problem

    def _evaluate(self, x, out, *args, **kwargs):
        # Evaluate the original problem
        res = {}
        self.problem._evaluate(x, res)

        # Combine original objectives with constraints as new objectives
        f = res["F"]
        # Constraint violation for g(x) > 0
        g = np.where(res["G"] > 0, res["G"], 0)
        h = np.abs(res["H"])  # Absolute value for equality constraints

        # Combine into a single objective array
        out["F"] = np.column_stack([f, g, h])

In [ ]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.selection.tournament import compare
from pymoo.optimize import minimize
from pymoo.problems import get_problem


class FeasibilityFirstNSGA2(NSGA2):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)

    def _next(self):
        # Overriding the selection process to prioritize feasibility
        def feasibility_first(a, b):
            # Check if either solution is feasible
            is_feasible_a = np.all(self.pop[a].get("F")[self.problem.n_obj :] <= 0)
            is_feasible_b = np.all(self.pop[b].get("F")[self.problem.n_obj :] <= 0)

            if is_feasible_a and not is_feasible_b:
                return a
            elif not is_feasible_a and is_feasible_b:
                return b
            else:
                # If both are feasible or infeasible, fall back to regular comparison
                return compare(
                    a, b, self.pop, self.n_survive, self.rank_key, self.crowding_key
                )

        # Tournament selection modified for feasibility
        mating_pool = self.selection(self.pop, self.n_offsprings, feasibility_first)
        off = self.crossover.do(self.problem, self.pop, mating_pool)
        off = self.mutation.do(self.problem, off)
        self.evaluator.eval(self.problem, off)
        self.pop = self.survival.do(
            self.problem, self.pop, off, n_survive=self.pop_size
        )


problem = get_problem("zdt1")
wrapped_problem = ConstraintAsObjectiveProblem(problem)

algorithm = FeasibilityFirstNSGA2(pop_size=100)

res = minimize(wrapped_problem, algorithm, ("n_gen", 100), verbose=True)